In [16]:
%run ./setting.ipynb

# configuration

In [17]:
weeks = 1

# load data

Nasdaq-100
INDEXNASDAQ: NDX

In [19]:
stock = yf.Ticker('NDX')

qqq = stock.history(
    period="max",
    interval = "1wk").reset_index()

qqq.to_sql(
    "qqq", 
    conn,
    if_exists = "replace",
    )

1970

In [21]:
df = sql(f"""
select * 
from qqq
order by Date
""")

fig = px.line(df, x='Date', y="Close")
fig.show()

In [36]:
sql(f"""
select 
a.*,
b.Close as Close_previouse,
b.Date as date_previouse,
a.Close/b.Close - 1 as return,
a.Close/b.Close as ratio
from qqq as a
left join qqq as b
on DATE(substr(b.Date, 1,10),'7 days') = DATE(substr(a.Date, 1,10))
""").to_sql(
    "qqq_with_previouse", 
    conn,
    if_exists = "replace",
    )

1970

# plot

In [38]:
df = sql(f"""
select 
Date,
Close,
'current' as type
from qqq_with_previouse
union 
select 
Date,
Close_previouse as Close,
'{weeks}-week ago' as type
from qqq_with_previouse
""")

In [39]:
fig = px.line(
    df, x="Date", y="Close", color='type', 
    )
fig.show()

# ratio

In [40]:
df_ratio = sql(f"""
select *
from qqq_with_previouse
""")

In [42]:
fig = px.line(
    df_ratio, x="Date", y="ratio",
    )
fig.show()

# box plot

In [43]:
fig = px.box(
    df_ratio, 
    y="ratio",
    )

fig.show()

# histogram 

In [44]:
fig = px.histogram(
    df_ratio, 
    x="ratio",
    nbins = 100,
    )
fig.show()

In [45]:
df_ratio[['ratio', 'return']].median()

ratio     1.004555
return    0.004555
dtype: float64

# end